# Converting National Elevation Dataset image files to Cloud Optimized GeoTIFFs (1/3rd arc, 1/9th arc, 1m)

## Required:
 - NED image files (as .img and .adf)
     - A globebd list for each 1/3rd arc, 1/9th arc and 1m
 - GDAL Translate command line function
 
## Workflow

 1. Define the function that will be used to write the .cmd file
 2.  Glob for a list of all image files in 1/3rd arc folder, 1/9th arc folder, and 1m arc folder
 3.  Specify the location to write the .cmd file to
     - Easy to write it to the same folder that the gdal tool is located in
 4.  Write the .cmd file for each list
 5. Open cmd prompt, cd to the location of the folder containing both the gdal tool and the .cmd file, run the .cmd file


In [1]:
# Import necessary modules
import os
from glob import glob

### Workflow step 1. 
#### Define the function that will be used to write the .cmd file

In [2]:
new_file_dir = r'T:\CCSI\TECH\FEMA\2017_NFIP_RiskRating_Milliman\GIS\Harddrive\NED\NED_COGTIFFS'
gdal_translate = 'gdal_translate'
gdal_warp = 'gdal_warp'

def WriteCMD(cmd_file, raster_paths, file_type, gdal_function, new_file_dir):
    '''This function opens a .cmd file in write mode, goes through a list of rasters and writes a command to use the gdal translate
    tool on each raster. This tool converts the raster to a cloud optimized GeoTIFF and puts each GeoTIFF in a common folder, 
    'new_file_dir'. The new GeoTIFFs are saved using their same file name from before conversion.'''
    quotes = '"'
    with open(cmd_file, 'w') as f:
        for img in raster_paths: 
            cmd = '{} '.format(gdal_function)
            gdal_options = ' -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW' # This formats the output GeoTIFF so that it is cloud optimized
            if '.img' in img:
                raw_file_name = os.path.basename(img).strip(file_type)
            else:
                raw_file_name = img.split('\\')[-2]
            new_file_name = os.path.join(new_file_dir, raw_file_name + '.tif')
            terminal_input = cmd + quotes + img +quotes+ ' '+ quotes + new_file_name + quotes + gdal_options + '\n' 
            f.write(terminal_input) # Writes a new line for each image file in the list
    return None

### Workflow step 2.
#### Glob for a lsit of all image files in 1/3rd arc folder, 1/9th arc folder, and 1m arc folder

In [3]:
# 1/3rd arc:
raster_dir_13 = r'T:\CCSI\TECH\FEMA\2017_NFIP_RiskRating_Milliman\GIS\Harddrive\NED\NED-1-3'
file_type_13 = '.adf'
adf_files = glob(os.path.join(raster_dir_13, '**', '*{}'.format(file_type_13)), recursive=True)

# Two .adf files for each panel, only using the one not ending in x
adf_imgs = [img for img in adf_files if 'w001001' in img]
adfs_13 = [img for img in adf_imgs if 'w001001x' not in img]

print('{} files with adf extension, {} 1/3rd arc raster files'.format(len(adf_files), len(adf_imgs)))

7300 files with adf extension, 2432 1/3rd arc raster files


In [4]:
# 1/9th arc:
raster_dir_19 = r'T:\CCSI\TECH\FEMA\2017_NFIP_RiskRating_Milliman\GIS\Harddrive\NED\NED-1-9'
file_type_19 = '.img'
imgs_19 = glob(os.path.join(raster_dir_19, '**', '*{}'.format(file_type_19)), recursive=True)

print('{} 1/9th arc raster files'.format(len(imgs_19)))

8354 1/9th arc raster files


In [5]:
# 1m arc:
raster_dir_1m = r'T:\CCSI\TECH\FEMA\2017_NFIP_RiskRating_Milliman\GIS\Harddrive\NED\NED-1m'
file_type_1m = '.img'
imgs_1m = glob(os.path.join(raster_dir_1m, '**', '*{}'.format(file_type_1m)), recursive=True)

print('{} 1m raster files'.format(len(imgs_1m)))

3642 1m raster files


### Workflow step 3.
#### Specify the location to write the .cmd file to

In [6]:
cmd_file_13 = r'C:\Users\abrazeau\Desktop\13_processing\RunBatch.cmd'
cmd_file_19 = r'C:\Users\abrazeau\Desktop\19_processing\RunBatch.cmd'
cmd_file_1m = r'C:\Users\abrazeau\Desktop\1m_processing\RunBatch.cmd'

### Workflow step 4.
#### Write the .cmd file for each list of images

In [7]:
# Write .cmd file for 1/3rd arc
WriteCMD(cmd_file_13, adfs_13, file_type_13, gdal_translate, new_file_dir)

print ('1/3rd arc .cmd file located at {}'.format(cmd_file_13))

1/3rd arc .cmd file located at C:\Users\abrazeau\Desktop\13_processing\RunBatch.cmd


In [8]:
# Write .cmd file for 1/9thrd arc
WriteCMD(cmd_file_19, imgs_19, file_type_19, gdal_translate, new_file_dir)

print ('1/9th arc .cmd file located at {}'.format(cmd_file_19))

1/9th arc .cmd file located at C:\Users\abrazeau\Desktop\19_processing\RunBatch.cmd


In [9]:
# Write .cmd file for 1m arc
WriteCMD(cmd_file_1m, imgs_1m, file_type_1m, gdal_translate, new_file_dir)

print ('1m arc .cmd file located at {}'.format(cmd_file_1m))

1m arc .cmd file located at C:\Users\abrazeau\Desktop\1m_processing\RunBatch.cmd


### Workflow step 5. 
#### Open cmd prompt, cd to the location of the folder containing both the gdal tool and the .cmd file, run the .cmd file


## References

##### gdal_translate information at: http://www.gdal.org/gdal_translate.html
##### Cloud Optimized GeoTIFF information at: https://trac.osgeo.org/gdal/wiki/CloudOptimizedGeoTIFF#HowtoreaditwithGDAL
##### 'Start' at the beginning of each line returns an error, cannot run gdal_translate as batch... would also questionable crash the machine